## Implementing VAE using Keras ## 
Links:

https://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/

https://blog.keras.io/building-autoencoders-in-keras.html

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.datasets import mnist 
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
K.set_learning_phase(1) #set learning phase
import tensorflow as tf


m = 50
n_z = 2
n_epoch = 10


# Q(z|X) -- encoder
inputs = Input(shape=(784,))
h_q = Dense(512, activation='relu')(inputs)
mu = Dense(n_z, activation='linear')(h_q)
log_sigma = Dense(n_z, activation='linear')(h_q)

Using TensorFlow backend.


In [2]:
def sample_z(args):
    mu, log_sigma = args
    eps = K.random_normal(shape=(m, n_z), mean=0., stddev=1.)
    return mu + K.exp(log_sigma / 2) * eps


# Sample z ~ Q(z|X)
z = Lambda(sample_z, output_shape=(n_z,))([mu, log_sigma])

In [3]:
# P(X|z) -- decoder
decoder_hidden = Dense(512, activation='relu')
decoder_out = Dense(784, activation='sigmoid')

h_p = decoder_hidden(z)
outputs = decoder_out(h_p)

In [4]:
# Overall VAE model, for reconstruction and training
vae = Model(inputs, outputs)

# Encoder model, to encode input into latent variable
# We use the mean as the output as it is the center point, the representative of the gaussian
encoder = Model(inputs, mu)

# Generator model, generate new data given latent variable z
d_in = Input(shape=(n_z,))
d_h = decoder_hidden(d_in)
d_out = decoder_out(d_h)
decoder = Model(d_in, d_out)

In [5]:
def vae_loss(y_true, y_pred):
    """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
    # E[log P(X|z)]
    recon = K.sum(K.binary_crossentropy(y_pred, y_true), axis=1)
    # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
    kl = 0.5 * K.sum(K.exp(log_sigma) + K.square(mu) - 1. - log_sigma, axis=1)

    return recon + kl

In [6]:
(x_trainO,_),(x_testO,_)=mnist.load_data()
x_test=x_testO.reshape(len(x_testO),np.prod(x_testO.shape[1:]))
x_train=x_trainO.reshape(len(x_trainO),np.prod(x_trainO.shape[1:]))
train_size=int(len(x_train)*0.8)
valid_size=int(len(x_train)*0.2)

## Normalizing the data between zero and 1 ## 
x_train=x_train.astype('float32')/255.
x_test=x_test.astype('float32')/255.
print x_test.shape
print x_train.shape

(10000, 784)
(60000, 784)


In [7]:
from keras.callbacks import ModelCheckpoint

vae.compile(optimizer='adam', loss=vae_loss)


In [8]:
Path='saved_models/Ezzat/weights.best.mnist.'+'vae_'+'2Layers'
print Path
checkpoint=ModelCheckpoint(filepath=Path, 
                               verbose=1, save_best_only=True)
vae.fit(x_train, x_train, validation_data=(x_test, x_test),
                shuffle=True,  callbacks=[checkpoint], verbose=1, batch_size=50, epochs=50)

saved_models/Ezzat/weights.best.mnist.vae_2Layers
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 17s - loss: 182.7386 - val_loss: 168.4977
Epoch 2/50
60000/60000 [==============================] - 17s - loss: 166.2977 - val_loss: 164.5850
Epoch 3/50
60000/60000 [==============================] - 17s - loss: 162.9396 - val_loss: 161.5367
Epoch 4/50
60000/60000 [==============================] - 17s - loss: 160.1890 - val_loss: 159.3686
Epoch 5/50
60000/60000 [==============================] - 16s - loss: 158.1193 - val_loss: 157.5877
Epoch 6/50
60000/60000 [==============================] - 17s - loss: 156.7390 - val_loss: 156.5366
Epoch 7/50
60000/60000 [==============================] - 17s - loss: 155.6931 - val_loss: 155.5947
Epoch 8/50
60000/60000 [==============================] - 16s - loss: 154.8645 - val_loss: 155.0183
Epoch 9/50
60000/60000 [==============================] - 17s - loss: 154.2003 - val_loss: 154.3615


60000/60000 [==============================] - 17s - loss: 148.2581 - val_loss: 150.4262
Epoch 31/50
60000/60000 [==============================] - 17s - loss: 148.1343 - val_loss: 150.5153
Epoch 32/50
60000/60000 [==============================] - 17s - loss: 148.0018 - val_loss: 150.3202
Epoch 33/50
60000/60000 [==============================] - 17s - loss: 147.8469 - val_loss: 150.2180
Epoch 34/50
60000/60000 [==============================] - 17s - loss: 147.7337 - val_loss: 150.1214
Epoch 35/50
60000/60000 [==============================] - 16s - loss: 147.6349 - val_loss: 150.2298
Epoch 36/50
60000/60000 [==============================] - 16s - loss: 147.5095 - val_loss: 150.2918
Epoch 37/50
60000/60000 [==============================] - 17s - loss: 147.3797 - val_loss: 149.8474
Epoch 38/50
60000/60000 [==============================] - 17s - loss: 147.2788 - val_loss: 149.9439
Epoch 39/50
60000/60000 [==============================] - 16s - loss: 147.1371 - val_loss: 149.7129
Ep

In [ ]:
vae.load(path)